In [ ]:
#query for efi2 deficit for Sleep problems, using updated code for efi2 deficits
 DROP TABLE `yhcr-prd-phm-bia-core.CB_MYSPACE_MK.temp_cb_2172_efalls_sleep_problem`;
 CREATE OR REPLACE TABLE `yhcr-prd-phm-bia-core.CB_2172.cb_2172_efalls_sleep_problem` as (
#bring all snoemd and ctv3codes for care home cohort in primary into the cte
with a as (
  select
  person_id,
  ctv3code,
  snomedcode,
  dateevent
  from `yhcr-prd-phm-bia-core.CB_FDM_PrimaryCare.tbl_srcode`
  where person_id in(select person_id from `yhcr-prd-phm-bia-core.CB_2172.temp_cb_2172_ch_cohort_demographic`)),
  #join care home entry date from msater table
  a_master as (
    select
    a.*,
    j.first_episodestartdate
    from a
    left join `yhcr-prd-phm-bia-core.CB_2172.temp_cb_2172_ch_cohort_demographic` j on
    a.person_id = j.person_id),
#filter for Sleep problems snomed and ctv3code codes
  b as (
    select
    person_id,
    dateevent,
    case 
    when snomedcode in(select SNOMEDCT_CONCEPTID from `yhcr-prd-phm-bia-core.CB_LOOKUPS.tbl_EFI2_Codelist` where deficit = 'Sleep problems') AND dateevent < first_episodestartdate AND (datetime_diff(first_episodestartdate,dateevent,year) <= 5) then 'true' 
    when ctv3code in(select CTV3 from `yhcr-prd-phm-bia-core.CB_LOOKUPS.tbl_EFI2_Codelist` where deficit = 'Sleep problems') AND dateevent < first_episodestartdate AND (datetime_diff(first_episodestartdate,dateevent,year) <= 5) then 'true' end as sleep_problem
    from a_master)
    
    select
    distinct person_id,
    sleep_problem
    from b
    where sleep_problem is not null)
  



